In [1]:
import numpy as np
import pandas as pd
import random as rd

In [1]:
%run Postproseccing.ipynb

In [2]:
def read_problem():
    """This function takes a huge ass txt file an converts it to a bit smaller dictonary which is divided into a bit smaller dictionary so that the human can understand it all maybe
    
    
    """
    pass

In [11]:

#feasibility helper functions

def all_jobs_handled(solution,problem):
    """This function runs through the solution and checks that all nodes linked to a job is visited twice and only twice.
        If so, all jobs are handled, and this part of the feasibility check is complete

    Args:
        solution (list): _description_
        problem (_type_): _description_
    """
    
    sol, vessels,charge_list = solution
    
    jobs = problem["Jobs"].keys()
    
    visited_d = set()
    visited_p = set()
    
    for s in sol:
        
        if s == (0,0):
            continue
        
        node, binary = s
        if binary == 0:
            visited_d.add(node)
        elif binary == 1:
            visited_p.add(node)
            
    
    for job in jobs:
        if job not in visited_d or job not in visited_p:
            return False
    
    return True
    
def passenger_cap_check(solution,problem):
    #THIS CAN ONLY BE DONE AFTER THE TRIPS HAVE BEEN ASSIGNED VESSELS
    #Since we do this check when allocating the vessels we dont have to do it here.
    vesseltypes = problem["Vessels"]
    sol, vessels,charge_list = solution
    
    max_pass = max_passengers_onboard(sol, problem)
    
    feasible = True
    
    for i in range(len(vessels)):
        
        vessel_id = vessels[i]
        if vessel_id == 0:
            feasible = False
            return feasible, f'No feasible Vessel for Trip {i}' 
        _,_,_,_,passenger_cap = vesseltypes[vessel_id]
        
        if max_pass[i]>passenger_cap:
            feasible = False
            return feasible, f'Passenger capacity for Vessel {i} broken'
    
    return feasible, 'Passenger cap check complete'


def feasible_trips(solution,problem):
    ####THIS FUNCTION DOES NOT SEEM TO WORK AS INTENDED
    """This function checks if there exists a combination of the trips generated by the solver which adhere to the constraints of the problem.
    The constraints are:
        (1) A node must be visited for delivery before it is visited for pickup, i.e (node,0) must come before (node,1) in the combined list.
        (2) All nodes in a given trip must be visisted in the order they are given in the trip. There can be nodes from different trips visisted between the nodes in the given trip,
        but trip[i-1]<trip[i]<trip[i+1] by means of placement must always be true.

    Args:
        solution (_type_): The generates solution from the solver. The trips are split by (0,0) nodes
        problem (_type_): A dictionary containing the problem instance

    Returns:
        _type_: If combined list is feasible, return True, else return False
    """
    
    combined = []
    
    found = set()
    
    sol, vessels ,_ = solution
    
    trips = split_solution(sol)
    
    trips = trips[:-1]
    rd.shuffle(trips)
    
    for j in range(len(trips)):
        trip = trips[j]
    
           
        for i in range(len(trip)):
            node, binary = trip[i]
            

            if i == 0: #Only rule 1
            
                if binary == 0:
                    combined.insert(0,trip[i])
                elif binary == 1:
                    if (node,0) in found:
                        lowest1 = combined.index((node,0))
                        combined.insert(lowest1+1,trip[i])
                    else:
                        combined.insert(0,trip[i]) #PUT IT AS EARLY AS POSSIBLE
            else:
                prev_node = combined.index(trip[i-1]) #index of the previous element in the trip
                
                if binary == 0:
                
                    if (node,1) in found:
                
                        node_1 = combined.index((node,1))
                        if prev_node<node_1:
                            combined.insert(prev_node+1,trip[i])
                        else:
                            print(f'Node {(trip[i])} is visited for pickup before delivery')
                            return False, combined
                    else:
                        combined.insert(prev_node+1,trip[i])   
                elif binary == 1:
                    
                    if (node,0) in found:
                        node_0 = combined.index((node,0)) #index of the delivery node
            
                        if node_0<=prev_node: 
                            combined.insert(prev_node+1,trip[i])
                        else:
                            combined.insert(node_0+1,trip[i])
                    else:
                        combined.insert(prev_node+1,trip[i])
            found.add((node,binary))
            
    
    
    return True, combined                
            

In [12]:
def feasibility_check(solution,problem):
    """This function takes a solution and checks if it is feasible. For now the two things that can brake feasibility is that the vessels are overfilled or that not all jobs are handled.
  
    Args:
        solution (_type_): _description_
        problem (_type_): _description_
    """
    
    
    feasibility = True
    
    sol,vessels,feasbile_post_processing = solution
    
    #passenger_check, _ = passenger_cap_check(solution,problem)
    
    #feasibility = passenger_check
    
    #handled_jobs = all_jobs_handled(solution,problem)
    
    #feasibility = handled_jobs
    
    all_trips_feasible, _ = feasible_trips(solution,problem)
    
    feasibility = all_trips_feasible
    
    return (feasibility and feasbile_post_processing), f'Passenger check: {True}. All jobs handled: {True}. All trips feasible: {all_trips_feasible}. Post processing feasible: {feasbile_post_processing}'
    
    
    

In [13]:
def single_vessel_obj_func(trip,vessel_id,problem):
    preproseccing = problem['PreProseccing']
    vesseltypes = problem['Vessels']
    vessel = vesseltypes[vessel_id]
    trip = [(0,0)] + trip + [(0,0)]
    tour = [x[0] for x in trip]
    cost_per_km = vessel[2]
    fixed_cost = vessel[5]
    
    tourlen, _ = tourlength(tour,preproseccing)
    
    return fixed_cost+ (tourlen*cost_per_km)
    

    

In [14]:
def objective_function(solution,problem):
    """This functions takes a solution and calculates the cost of this solution after collection information from
    the problem-file. The costs consist of operational cost for the vessels (going from node to node), charging cost done at the charging stations and penalties for not handling jobs

    Args:
        solution (_type_): A tuple with the trips given by the solver with added charging, the vessel allocation and the list of charging done
        problem (_type_): A dictonary of information
        
    Return:
        cost: Return the total cost of the solution given 
    """
    
    cost_info = problem['Costs']
    jobs = problem['Jobs']
    operational_cost = 0
    penalties = 0
    #charging_cost = 0 #Going to charge leads to larger operation -> Not needed
    
    sol, vessels, charge_list = solution
    

    
    trips = split_solution(sol)
    

    for i in range(len(vessels)):
        vessel_id = vessels[i]
        if vessel_id == 0:
            continue

        single_cost = single_vessel_obj_func(trips[i],vessel_id,problem)
        operational_cost += single_cost
        
        
    
    
    outsourced = trips[-1]
    
    for o in outsourced:
        job_id, binary = o
        n_crew = jobs[job_id]
        penalties += cost_info["Fixed Penalty"]+ (cost_info["PenaltyPerCrew"]*n_crew)
    
    return sum([operational_cost,penalties])

In [15]:
def initial_solution(problem,n_trips): #Could maybe start with a shuffled list. By not shuffling we help the model keep the pairs together
    
    nodes = problem["Nodes"]
    
    sol = []
    
    for i in range(n_trips):
        sol.append((0,0))
        
    sol.extend(nodes)
    vessels = []
    charging_list = []

    print(f'Initial Solution:           {sol}')
    return sol, vessels, charging_list